In [1]:
!pip install geocoder

In [2]:
!pip install folium

# 1)Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# 2) Scraping Data From Wikipedia

In [4]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_districts_of_Istanbul")[0]

In [5]:
df.head()

,District,Population (2019),Area (km²),Density (per km²),Mensual household income TL(USD),Annual household income TL(USD)
0,Adalar,15238,11.05,1379,6.652₺ (918$),"79.821₺ (10,978$)"
1,Arnavutköy,282488,450.35,627,2.030₺ (279$),"24.360₺ (3,350$)"
2,Ataşehir,425094,25.23,16849,6.577₺ (904$),"78.924₺ (10,854$)"
3,Avcılar,448882,42.01,10685,3.662₺ (503$),"43.938₺ (6,064$)"
4,Bağcılar,745125,22.36,33324,3.197₺ (441$),"38.367₺ (5,295$)"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
District                            43 non-null object
Population (2019)                   43 non-null int64
Area (km²)                          43 non-null object
Density (per km²)                   43 non-null int64
Mensual household income TL(USD)    40 non-null object
Annual household income TL(USD)     40 non-null object
dtypes: int64(2), object(4)
memory usage: 2.1+ KB


In [7]:
IstDistricts = df[["District"]]

In [8]:
IstDistricts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 1 columns):
District    43 non-null object
dtypes: object(1)
memory usage: 424.0+ bytes


In [9]:
IstDistricts.head()

,District
0,Adalar
1,Arnavutköy
2,Ataşehir
3,Avcılar
4,Bağcılar


In [10]:
IstDistricts.tail()

,District
38,Zeytinburnu
39,Europe (25 districts)
40,Asia (14 districts)
41,Urban (36 districts)[a]
42,TOTAL


In [11]:
IstDistricts.drop([IstDistricts.index[39], IstDistricts.index[40], IstDistricts.index[41], IstDistricts.index[42]], inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
IstDistricts.shape

(39, 1)

# 3) Geographical Coordinates

In [13]:
#define a function to get coordinates
def get_latlng(district):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, istanbul, Turkey'.format(district))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
coords = [ get_latlng(district) for district in IstDistricts["District"].tolist() ]

In [15]:
coords

[[40.863060000000075, 29.12423000000007],
 [41.18558000000007, 28.74147000000005],
 [40.99248000000006, 29.127770000000055],
 [40.97813000000008, 28.721010000000035],
 [41.03323000000006, 28.863510000000076],
 [40.998050000000035, 28.84737000000007],
 [40.98273000000006, 28.87480000000005],
 [41.10745000000003, 28.800610000000063],
 [41.043810000000065, 28.91214000000008],
 [41.041900000000055, 29.005750000000035],
 [41.12647000000004, 29.097410000000025],
 [41.00579000000005, 28.65930000000003],
 [41.03672000000006, 28.986120000000028],
 [41.02046000000007, 28.587290000000053],
 [41.14369000000005, 28.460350000000062],
 [41.03290000000004, 29.173830000000066],
 [41.03932000000003, 28.881150000000048],
 [41.03246000000007, 28.660930000000064],
 [41.04657000000003, 28.931790000000035],
 [41.019580000000076, 28.94726000000003],
 [41.05728000000005, 28.91605000000004],
 [41.022050000000036, 28.878280000000075],
 [40.98867000000007, 29.02732000000003],
 [41.08011000000005, 28.9733200000000

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
IstDistricts['Latitude'] = df_coords['Latitude']
IstDistricts['Longitude'] = df_coords['Longitude']

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [18]:
IstDistricts

,District,Latitude,Longitude
0,Adalar,40.86306,29.12423
1,Arnavutköy,41.18558,28.74147
2,Ataşehir,40.99248,29.12777
3,Avcılar,40.97813,28.72101
4,Bağcılar,41.03323,28.86351
5,Bahçelievler,40.99805,28.84737
6,Bakırköy,40.98273,28.87480
7,Başakşehir,41.10745,28.80061
8,Bayrampaşa,41.04381,28.91214
9,Beşiktaş,41.04190,29.00575


In [20]:
IstDistricts.to_csv("IstDistricts.csv", index=False)

# 4)Creating a map of Istanbul with neighborhoods superimposed on top

In [21]:
# get the coordinates of Istanbul
address = 'Istanbul, Turkey'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of İstanbul, Turkey {}, {}.'.format(latitude, longitude))

The geograpical coordinate of İstanbul, Turkey 41.0096215, 28.9651867.


In [22]:
# create map of Istanbul using latitude and longitude values
map_Ist = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, dst in zip(IstDistricts['Latitude'], IstDistricts['Longitude'], IstDistricts['District']):
    label = '{}'.format(dst)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Ist)  
    
map_Ist

In [23]:
# save the map as HTML file
map_Ist.save('map_Ist.html')

# 5) Use the Foursquare API to explore the neighborhoods

In [24]:
CLIENT_ID = 'DLYQON1DN5F4EELF2WSYN10OZZLQC4JHAMVCM1VZNWKAJL1R'
CLIENT_SECRET = 'OMJDQWKBXLZQIOP2QDFLSFXA5AQJ0ZPEPBZIL1OTXYRA0545'
VERSION = '20200623'

In [26]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, dst in zip(IstDistricts['Latitude'], IstDistricts['Longitude'], IstDistricts['District']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            dst,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['District', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3838, 7)


,District,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adalar,40.86306,29.12423,Büyükada Lale köşkü,40.865657,29.125223,Bed & Breakfast
1,Adalar,40.86306,29.12423,Büyükada Tepesi,40.861107,29.117418,Mountain
2,Adalar,40.86306,29.12423,Eski Rum Yetimhanesi,40.861705,29.123323,Historic Site
3,Adalar,40.86306,29.12423,BüyükAdA Loc'AdA,40.857548,29.126157,Restaurant
4,Adalar,40.86306,29.12423,Büyükada Bisiklet Parkuru,40.865000,29.116861,Bike Trail


In [29]:
#Let's check how many venues were returned for each neighorhood
venues_df.groupby(["District"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
District,,,,,,
Adalar,100,100,100,100,100,100
Arnavutköy,46,46,46,46,46,46
Ataşehir,100,100,100,100,100,100
Avcılar,100,100,100,100,100,100
Bahçelievler,100,100,100,100,100,100
Bakırköy,100,100,100,100,100,100
Bayrampaşa,100,100,100,100,100,100
Bağcılar,100,100,100,100,100,100
Başakşehir,100,100,100,100,100,100


In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 301 uniques categories.


In [31]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bed & Breakfast', 'Mountain', 'Historic Site', 'Restaurant',
       'Bike Trail', 'Hotel', 'Beach', 'Garden', 'Café', 'Motel',
       'Rest Area', 'Bistro', 'Dessert Shop', 'Breakfast Spot',
       'Ice Cream Shop', 'Art Gallery', 'Campground',
       'Australian Restaurant', 'Hot Dog Joint', 'National Park',
       'Scenic Lookout', 'Hotel Bar', 'Waterfront', 'Gourmet Shop',
       'History Museum', 'Entertainment Service', 'Seafood Restaurant',
       'Plaza', 'Turkish Restaurant', 'Event Space', 'Kofte Place', 'Bar',
       'Mediterranean Restaurant', 'Diner', 'Road', 'Water Park',
       'Tea Room', 'BBQ Joint', 'Gym', 'Electronics Store', 'Bakery',
       'Gym / Fitness Center', 'Fast Food Restaurant', 'Cafeteria',
       'Arcade', 'Hookah Bar', 'Turkish Home Cooking Restaurant',
       'Pharmacy', 'Fish & Chips Shop', 'Buffet'], dtype=object)

# 6) Analyze Each Neighborhood

In [32]:
# one hot encoding
Ist_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add districts column back to dataframe
Ist_onehot['Districts'] = venues_df['District'] 

# move neighborhood column to the first column
fixed_columns = [Ist_onehot.columns[-1]] + list(Ist_onehot.columns[:-1])
Ist_onehot = Ist_onehot[fixed_columns]

print(Ist_onehot.shape)
Ist_onehot.head()

(3838, 302)


,Districts,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Trail,Bistro,Boarding House,Bookstore,Borek Place,Bosnian Restaurant,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Business Center,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Escape Room,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Gozleme Place,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,Ice Cream Shop,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Kids Store,Kofte Place,Kokoreç Restaurant,Kuruyemişçi,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Manti Place,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Middle Eastern Restaurant,Military Base,Mobile Phone Shop,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Nightclub,Nightlife Spot,Non-Profit,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pilavcı,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Shrine,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tantuni Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Transportation Service,Travel Agency,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Watch Shop,Water Park,Waterfront,Wedding Hall,Wine Ba

In [33]:
Ist_grouped = Ist_onehot.groupby(["Districts"]).mean().reset_index()

print(Ist_grouped.shape)
Ist_grouped

(39, 302)


,Districts,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Trail,Bistro,Boarding House,Bookstore,Borek Place,Bosnian Restaurant,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Business Center,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Caribbean Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Escape Room,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Gozleme Place,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Housing Development,Hunting Supply,Ice Cream Shop,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Kids Store,Kofte Place,Kokoreç Restaurant,Kuruyemişçi,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Manti Place,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meyhane,Middle Eastern Restaurant,Military Base,Mobile Phone Shop,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Nightclub,Nightlife Spot,Non-Profit,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pilavcı,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Shrine,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tantuni Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Transportation Service,Travel Agency,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Watch Shop,Water Park,Waterfront,Wedding Hall,Wine Ba

In [37]:
len(Ist_grouped[Ist_grouped["Turkish Restaurant"] > 0])

35

In [39]:
Ist_restaurants = Ist_grouped[["Districts","Turkish Restaurant"]]

In [40]:
Ist_restaurants.head()

,Districts,Turkish Restaurant
0,Adalar,0.020000
1,Arnavutköy,0.086957
2,Ataşehir,0.000000
3,Avcılar,0.010000
4,Bahçelievler,0.040000


# 7) Cluster Neighborhoods

In [41]:
# set number of clusters
kclusters = 3

Ist_clustering = Ist_restaurants.drop(["Districts"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Ist_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 0, 2, 1, 1, 1, 2, 0], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Ist_merged = Ist_mall.copy()

# add clustering labels
Ist_merged["Cluster Labels"] = kmeans.labels_

In [43]:
Ist_merged.rename(columns={"Districts": "District"}, inplace=True)
Ist_merged.head()

,District,Turkish Restaurant,Cluster Labels
0,Adalar,0.020000,0
1,Arnavutköy,0.086957,1
2,Ataşehir,0.000000,0
3,Avcılar,0.010000,0
4,Bahçelievler,0.040000,2


In [45]:
# merge Istanbul_grouped with Istanbul_data to add latitude/longitude for each neighborhood
Ist_merged = Ist_merged.join(IstDistricts.set_index("District"), on="District")

print(Ist_merged.shape)
Ist_merged.head() # check the last columns!

(39, 5)


,District,Turkish Restaurant,Cluster Labels,Latitude,Longitude
0,Adalar,0.020000,0,40.86306,29.12423
1,Arnavutköy,0.086957,1,41.18558,28.74147
2,Ataşehir,0.000000,0,40.99248,29.12777
3,Avcılar,0.010000,0,40.97813,28.72101
4,Bahçelievler,0.040000,2,40.99805,28.84737


In [46]:
# sort the results by Cluster Labels
print(Ist_merged.shape)
Ist_merged.sort_values(["Cluster Labels"], inplace=True)
Ist_merged

(39, 5)


,District,Turkish Restaurant,Cluster Labels,Latitude,Longitude
0,Adalar,0.020000,0,40.86306,29.12423
31,Tuzla,0.000000,0,40.81753,29.30411
26,Sancaktepe,0.020000,0,40.98593,29.22865
20,Kadıköy,0.000000,0,40.98867,29.02732
37,Şile,0.000000,0,41.17646,29.61227
12,Beşiktaş,0.020000,0,41.04190,29.00575
11,Beyoğlu,0.010000,0,41.03672,28.98612
9,Beykoz,0.020000,0,41.12647,29.09741
38,Şişli,0.010000,0,41.06201,28.98877
2,Ataşehir,0.000000,0,40.99248,29.12777


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Ist_merged['Latitude'], Ist_merged['Longitude'], Ist_merged['District'], Ist_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# 8) Examine Clusters

In [52]:
Ist_merged.loc[Ist_merged['Cluster Labels'] == 0]

,District,Turkish Restaurant,Cluster Labels,Latitude,Longitude
0,Adalar,0.02,0,40.86306,29.12423
31,Tuzla,0.00,0,40.81753,29.30411
26,Sancaktepe,0.02,0,40.98593,29.22865
20,Kadıköy,0.00,0,40.98867,29.02732
37,Şile,0.00,0,41.17646,29.61227
12,Beşiktaş,0.02,0,41.04190,29.00575
11,Beyoğlu,0.01,0,41.03672,28.98612
9,Beykoz,0.02,0,41.12647,29.09741
38,Şişli,0.01,0,41.06201,28.98877
2,Ataşehir,0.00,0,40.99248,29.12777


In [54]:
Ist_merged.loc[Ist_merged['Cluster Labels'] == 1]

,District,Turkish Restaurant,Cluster Labels,Latitude,Longitude
7,Bağcılar,0.070000,1,41.03323,28.86351
36,Üsküdar,0.070000,1,41.02733,29.01566
34,Çekmeköy,0.070000,1,41.03290,29.17383
33,Çatalca,0.065217,1,41.14369,28.46035
32,Zeytinburnu,0.080000,1,40.99526,28.90170
1,Arnavutköy,0.086957,1,41.18558,28.74147
30,Sultangazi,0.070000,1,41.09816,28.90358
29,Sultanbeyli,0.080000,1,40.96757,29.27080
22,Kağıthane,0.080000,1,41.08011,28.97332
21,Kartal,0.070000,1,40.89064,29.19082


In [55]:
Ist_merged.loc[Ist_merged['Cluster Labels'] == 2]

,District,Turkish Restaurant,Cluster Labels,Latitude,Longitude
27,Sarıyer,0.04,2,41.15998,29.04180
28,Silivri,0.04,2,41.07620,28.24460
16,Eyüpsultan,0.04,2,41.04657,28.93179
4,Bahçelievler,0.04,2,40.99805,28.84737
14,Esenler,0.05,2,41.03932,28.88115
25,Pendik,0.03,2,40.87624,29.23432
24,Maltepe,0.04,2,40.92496,29.13087
8,Başakşehir,0.05,2,41.10745,28.80061
35,Ümraniye,0.05,2,41.01643,29.12476
10,Beylikdüzü,0.05,2,41.00579,28.65930


# Observations:

Most of the Turkish restaurants are concentrated in the central area of Istanbul city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number. This represents a great opportunity and high potential areas to open new Turkish restaurants as there is very little to no competition from existing restaurants. Meanwhile, Turkish restaurants in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of Turkish restaurants. From another perspective, this also shows that the oversupply of Turkish restaurants mostly happened in the central area of the city, with the suburb area still have very few restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Turkish restaurants in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.